## Cortical structure experiments - 2

This script performs permutation testing experiments on cortical structures betweeen diagnostic groups for each presentation.
This script generates the necessary files to run the freesurfer glmfit -sim command, with permutation testing, and save the results, for later analysis and visualization.

More info about the permutation procedure in freesurfer can be found in:
https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/MultipleComparisonsV6.0Perm

In [3]:
import numpy as np
import pandas as pd
import os

**Initialization and loading of variables**

In [4]:
# Parameters of the procedure
clusters = 4
rd_seed = 1714
n_perm = 1000

# thickness or volume
mode = "thickness"
DX = [('CN','AD'), ('CN', 'LMCI'), ('LMCI', 'AD')]
C = ["C1", "C2", "C3", "C4"]

# Path of the generated clusters, after a run of cimlr-ad.py
cluster_path = "/homedtic/gmarti/EXPERIMENTS/CIMLR-AD/cimlr4/"      
df_cluster = pd.read_csv(cluster_path + 'cluster_data.csv')
df_cluster.reset_index(inplace=True)

**Prepare files for the concrete problem**

In [5]:
# Names of subjects
fsid_names = [x for x in map(lambda s: 'ADNI' + s[0:3] + 'S' + s[6:], 
                             df_cluster.PTID.values)]

# Support dataframe, with the info that we need to write
df_qdec = pd.DataFrame(data={'fsid': fsid_names,
                            'cluster': df_cluster.C.values,
                            'diagnosis': df_cluster.DX.values},
                       columns = ['fsid', 'cluster', 'diagnosis'])

# We want to test the thickness in the population of each cluster
# against the rest, so that we can observe differences across the 
# whole cluster.

# Contrast: same for all the 4 experiments
def write_contrast(name, matrix):
    with open(name, 'w') as f:
        f.write(matrix)  # First line, mandatory

# Create a fsgd file for each of the tests.
fsgd_files = []
for d in DX:
    for c in C:
        # Create FSGD file
        # Only write the subjects that 
        fsgd_file = 'freefiles/adni_files_ex2_' + c + '_' + d[0] + '_' + d[1] + '.fsgd'
        fsgd_files.append(fsgd_file)
        with open(fsgd_file, 'w') as f:
            f.write('GroupDescriptorFile 1\n')  # First line, mandatory
            # Title, optional
            f.write('Title ex2-c ' + c + ' ' + d[0] + 'vs' +  d[1] + '\n')
            # three classes, two corresponding to diagnosis and rest
            # rest will not be considered for test, but need to be included
            # for later testing
            # rest will be CN and AD classes that are 
            f.write('Class ' + d[0] + '\n')
            f.write('Class ' + d[1] + '\n')
            f.write('Class Rest\n')   # Title, optional

            # List to store diagnosis group (either 0 or 1) for later group permutation
            eb_list = []
            for subj in df_qdec.itertuples():
                subj_ptid = subj.fsid
                # Select only subjects in the cluster we are selecting
                # For all the subjects with the two diagnosis, assign them to class if they
                # are included in the cluster, and to Rest if not.
                if d[0] == subj.diagnosis:
                    eb_list.append(0)
                    if c == 'C' + str(int(subj.cluster)):
                        cluster = d[0]
                    else:
                        cluster = 'Rest'
                elif d[1] == subj.diagnosis:
                    eb_list.append(1)
                    if c == 'C' + str(int(subj.cluster)):
                        cluster = d[1]
                    else:
                        cluster = 'Rest'
                else:
                    continue
                f.write('Input ' + subj_ptid + ' ' + cluster + '\n')

            # NEED TO ALSO CREATE EB FILE
            eb_out = 'freefiles/adni_files_ex2_' + c + '_' + d[0] + '_' + d[1] + '.csv'
            with open(eb_out, 'w') as out:
                for n in eb_list:
                    out.write(str(n) + '\n')
                    
# We will have a contrast file with 3 columns. Last one is 0!
mtx = 'freefiles/ex2.mtx'
contrast = "+1 -1 0\n"
write_contrast(mtx, contrast)

**Run mri-prepoc with the selected subjects**

In [6]:
# For each experiment, run mri-preproc to prepare for the experiments.
# config variables:

# fsgd files contain all the experiments already, so only one loop is needed
# also, need to do lh and rh hemisferes
hemis = ["lh", "rh"]
for f in fsgd_files:
    name = os.path.splitext(os.path.basename(f))[0]
    for h in hemis:
        # build the command
        cmdline = ["mris_preproc", '--fsgd', f]
        # fwhm10 hard coded! (TODO: maybe don't hardcode this?)
        cmdline += ['--cache-in', mode + '.fwhm10.fsaverage']
        cmdline += ['--target', 'fsaverage']
        cmdline += ['--hemi', h]
        cmdline += ['--out', "processed_files/" + h + "_" + mode + "." + name + ".10.mgh"]
        # If out file exists, no need to run it again!
        if os.path.exists("processed_files/" + h + "_" + mode + "." + name + ".10.mgh"):
            print("processed_files/" + h + "_" + mode + "." + name + ".10.mgh Already exists!")
            continue
        else:
            # Run the experiment
            os.system(' '.join(cmdline))

processed_files/lh_thickness.adni_files_ex2_C1_CN_AD.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex2_C1_CN_AD.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex2_C2_CN_AD.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex2_C2_CN_AD.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex2_C3_CN_AD.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex2_C3_CN_AD.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex2_C4_CN_AD.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex2_C4_CN_AD.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex2_C1_CN_LMCI.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex2_C1_CN_LMCI.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex2_C2_CN_LMCI.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex2_C2_CN_LMCI.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex2_C3_CN_LMCI.10.mgh Already exists!
pr

**run mri-glmfit and correct for multiple comparisons**

In [7]:
# Again, no need to do another loop
hemis = ["lh", "rh"]
for f in fsgd_files:
    name = os.path.splitext(os.path.basename(f))[0]
    for h in hemis:
        # build the command
        cmdline = ["mri_glmfit"]
        cmdline += ['--y', "processed_files/" + h + "_" + mode + "." + name + ".10.mgh"]
        cmdline += ['--fsgd', f, 'dods']
        cmdline += ['--C', mtx]
        cmdline += ['--surf', 'fsaverage', h]
        cmdline += ['--cortex']
        cmdline += ['--glmdir', "glm_output/" + h + "_" + name + ".glmdir"]
        # Needed for later correction
        cmdline += ['--eres-save']
        
        # Run the experiment
        os.system(' '.join(cmdline))
        

In [ ]:
# RUN PALM FOR MULTIPLE COMPARISONS
## Correction for multiple comparisons
hemis = ["lh", "rh"]
for f in fsgd_files:
    name = os.path.splitext(os.path.basename(f))[0]
    for h in hemis:
        # build the command
        cmdline = ["fspalm"]
        cmdline += ['--glmdir', "glm_output/" + h + "_" + name + ".glmdir"]
        cmdline += ['--cft', '1.3']
        cmdline += ['--twotail']
        cmdline += ['--name', 'palm_' + h + "_" + name]
        cmdline += ['--iters', '1000']
        cmdline += ['--2spaces']
        cmdline += ['--cwp', '.05']
        cmdline += ['--pargs="' + 'eb=freefiles/' + name + '.csv"']

        os.system(' '.join(cmdline))